In [16]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Fetched 338 kB in 1s (290 kB/s)
Reading package lists... Done


In [40]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [41]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [42]:
# created a df to view output before temp made
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [43]:
#find values for each column
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [44]:
#Import year function to seperate year
from pyspark.sql.functions import month, year

#Seperate year
df.select(year(df["date"]))

#create a new column in df for year only
df = df.withColumn("year", year(df["date"]))

df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
+--------------------+----------+-------

In [45]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- year: integer (nullable = true)



In [46]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [47]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT year,  ROUND(AVG(price),2) AS avg_price
FROM home_sales
WHERE bedrooms = '4'
GROUP BY year
"""

spark.sql(query).show(5)

+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [48]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT date_built,  ROUND(AVG(price),2) AS avg_price
FROM home_sales
WHERE bedrooms = '3'
AND bathrooms = '3'
GROUP BY date_built
"""

spark.sql(query).show(5)


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
+----------+---------+
only showing top 5 rows



In [49]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT date_built,  ROUND(AVG(price),2) AS avg_price
FROM home_sales
WHERE bedrooms = '3'
AND bathrooms = '3'
AND floors = '2'
AND sqft_living >= '2000'
GROUP BY date_built
"""

spark.sql(query).show(5)


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
+----------+---------+
only showing top 5 rows



In [50]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT view,  ROUND(AVG(price),2) AS avg_price
FROM home_sales
WHERE price >= '350000'
GROUP BY view
"""

spark.sql(query).show(5)


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
+----+----------+
only showing top 5 rows

--- 0.3648958206176758 seconds ---


In [51]:
# 7. Cache the the temporary table sales.
spark.catalog.cacheTable("home_sales")

In [52]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [53]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT view,  ROUND(AVG(price),2) AS avg_price
FROM home_sales
WHERE price >= '350000'
GROUP BY view
"""

spark.sql(query).show(5)


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
+----+----------+
only showing top 5 rows

--- 1.2335479259490967 seconds ---


In [54]:
#write out data in parquet format activity 1,3
df.write.parquet('parquet_sales', mode='overwrite')

In [55]:
#read in our new parquet format step 7 activity 1,3
p_df=spark.read.parquet('parquet_sales')

In [56]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data activity 3
p_df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_sales_built")

In [57]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('parquet_sales_built')

In [58]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView("p_sales")

In [59]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query = """
SELECT view,  ROUND(AVG(price),2) AS avg_price
FROM p_sales
WHERE price >= '350000'
GROUP BY view
"""

spark.sql(query).show(5)


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
+----+----------+
only showing top 5 rows

--- 0.7242169380187988 seconds ---


In [60]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [61]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

#verify if cached or not
if is_cached:
   print("Table 'home_sales' is still cached.")
else:
   print("Table 'home_sales' has been successfully uncached.")

Table 'home_sales' has been successfully uncached.
